# Summarize results across assays
This notebook makes summarizes the results across assays.

In [ ]:
import functools
import operator
import re

import altair as alt

import pandas as pd

import polyclonal.alphabets
from polyclonal.plot import color_gradient_hex

_ = alt.data_transformers.disable_max_rows()

The next cell is tagged as `parameters` for `papermill` parameterization:

In [ ]:
site_numbering_map_csv = None
chart_overlaid = None
chart_faceted = None
csv_file = None
config = None
input_csvs = None

In [ ]:
# Parameters
config = {
    "min_times_seen": 3,
    "min_frac_models": 1,
    "alphabet": [
        "A",
        "C",
        "D",
        "E",
        "F",
        "G",
        "H",
        "I",
        "K",
        "L",
        "M",
        "N",
        "P",
        "Q",
        "R",
        "S",
        "T",
        "V",
        "W",
        "Y",
        "-",
    ],
    "init_floor_escape_at_zero": True,
    "init_site_escape_stat": "mean",
    "antibody_escape": {
        "monoclonal antibodies": {
            "stat": "escape_median",
            "negative_color": "#0072B2",
            "positive_color": "#E69F00",
            "max_at_least": 1,
            "min_at_least": -1,
            "antibody_list": ["REGN10933", "S2M11"],
        }
    },
    "func_effects": {
        "spike-mediated infection": {
            "condition": "293T_ACE2_entry",
            "effect_type": "func_effects",
            "positive_color": "#009E73",
            "negative_color": "#CC79A7",
            "max_at_least": 1,
            "min_at_least": 0,
        }
    },
    "other_assays": {
        "receptor_affinity": {
            "mock receptor affinity": {
                "condition": "pretending_S2M11_is_receptor",
                "stat": "receptor affinity_median",
                "positive_color": "#FF715B",
                "negative_color": "#F3C13A",
                "max_at_least": 1,
                "min_at_least": 0,
            }
        }
    },
}
input_csvs = {
    "antibody_escape REGN10933": "results/antibody_escape/averages/REGN10933_mut_effect.csv",
    "antibody_escape S2M11": "results/antibody_escape/averages/S2M11_mut_effect.csv",
    "func_effects 293T_ACE2_entry": "results/func_effects/averages/293T_ACE2_entry_func_effects.csv",
    "receptor_affinity pretending_S2M11_is_receptor": "results/receptor_affinity/averages/pretending_S2M11_is_receptor_mut_effect.csv",
}
site_numbering_map_csv = "data/site_numbering_map.csv"
chart_faceted = "results/summaries/summary_faceted_nolegend.html"
chart_overlaid = "results/summaries/summary_overlaid_nolegend.html"
csv_file = "results/summaries/summary.csv"
